In [2]:
from netCDF4 import Dataset
import numpy as np
import os
from datetime import datetime, timedelta
import pandas as pd

In [3]:
def get_per_time_slot_values(prcp):
    per_interval_prcp = (prcp[1:] - prcp[:-1])
    return per_interval_prcp    

In [ ]:
def read_netcdf(netcdf_file):
    nnc_fid = Dataset(rainnc_net_cdf_file, mode='r')
    time_unit_info = nnc_fid.variables['XTIME'].units
    print('time_unit_info : ', time_unit_info)
    time_unit_info_list = time_unit_info.split('since ')
    print(time_unit_info_list)

    lats = nnc_fid.variables['XLAT'][0, :, 0]
    lons = nnc_fid.variables['XLONG'][0, 0, :]

    lon_min = lons[0].item()
    lat_min = lats[0].item()
    lon_max = lons[-1].item()
    lat_max = lats[-1].item()

    lat_inds = np.where((lats >= lat_min) & (lats <= lat_max))
    lon_inds = np.where((lons >= lon_min) & (lons <= lon_max))

    rainnc = nnc_fid.variables['RAINNC'][:, lat_inds[0], lon_inds[0]]
    times = nnc_fid.variables['XTIME'][:]
    nnc_fid.close()
    
    diff = get_per_time_slot_values(rainnc)
    width = len(lons)
    height = len(lats)
    
    ts_df_list = []
    
    for i in range(len(diff)):
        ts_time = datetime.strptime(time_unit_info_list[1], '%Y-%m-%d %H:%M:%S') + 
            timedelta(minutes=times[i + 1].item())
        t = datetime_utc_to_lk(ts_time, shift_mins=0)
        timestamp = t.strftime('%Y-%m-%d_%H-%M')
        ts_list = []
        for y in range(height):
            for x in range(width):
                lat = float('%.6f' % lats[y])
                lon = float('%.6f' % lons[x])
                rain = '%.3f' % float(diff[i, y, x])
                ts_list.append([lon, lat, rain])
        ts_df = pd.DataFrame(ts_list, columns=['Lon', 'Lat', 'Rain'])
        ts_df_list.append(ts_df)
    return ts_df_list